# Linear Regression Implementation

### 1. Linear Regression

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

plt.style.use('seaborn')
%matplotlib inline

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/justmarkham/scikit-learn-videos/master/data/Advertising.csv')
df

In [ ]:
# prepare X (features) and y (target)
feature_cols = ['TV', 'Newspaper', 'Radio']
X = df[feature_cols]
y = df['Sales']

In [ ]:
scaler = StandardScaler()

# do train_test_split (train:75, test:25)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# do scaling for training convergence
X_train, X_test = scaler.fit_transform(X_train), scaler.fit_transform(X_test)

In [ ]:
# determine whether system supports CUDA or not 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# convert numpy arrays into pytorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test.values, dtype=torch.float32).to(device)

In [ ]:
cost = []

class LinearRegression(torch.nn.Module):
    
    def __init__(self, num_features):
        super(LinearRegression, self).__init__()       
        self.num_features = num_features
        self.linear = torch.nn.Linear(num_features, 1)
        self.linear.weight.detach().normal_(0, .1)
        self.linear.bias.detach().zero_()
        
    
    def forward(self, x):
        logits = self.linear(x)
        yhat = logits.view(-1)
        return yhat
    

    def train(self, x, y, num_epochs, learning_rate=0.1):
        
        # use gradient descent as the optimizer
        optimizer = torch.optim.SGD(self.parameters(), lr=learning_rate)
        
        for e in range(num_epochs):
            
            # compute outputs ###
            yhat = self.forward(x)
            
            # compute the loss
            loss = F.mse_loss(yhat, y, reduction='mean')
            
            # reset gradients from the previous interaction
            optimizer.zero_grad()
            
            # comp. gradients
            loss.backward()          
            
            # update weights and bias
            optimizer.step()

            
            ### Logging ####
            with torch.no_grad():
                yhat = self.forward(x)
                curr_loss = F.mse_loss(yhat, y, reduction='mean')
                print('Epoch: %03d' %(e+1), end='')
                print(' | MSE: %.3f' %curr_loss)
                cost.append(curr_loss) 
            
    def predict(self, x):
        predictions = self.forward(x)
        return predictions

In [ ]:
# training
linreg = LinearRegression(num_features=X_train.size(1))
linreg.to(device)
linreg.train(X_train, y_train, num_epochs=70)

In [ ]:
# plot the MSE-loss after each epoch
plt.plot(range(len(cost)), cost)
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.show()

In [ ]:
# examine the coefficients and intercept
print(f'Coefficients: {list(zip(feature_cols, linreg.linear.weight.detach()))}')
print(f'Intercept: {linreg.linear.bias.detach()}')
print(f'- MSE: {F.mse_loss(y_test, linreg.predict(X_test).detach()):.3f}')

**Use scikit-learn**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)

print(f'Coefficients: {list(zip(feature_cols, linreg.coef_))}')
print(f'Intercept: {linreg.intercept_:.4f}')
print(f'- MSE: {mean_squared_error(y_test, y_pred):.3f}')

**Analytical solution**
$$(X^T.X)^{-1}.X^T.y$$
- Set: $$a = (X^T.X)^{-1}, b=a.X^T, w=b.y$$
- After finding w, we can compute the bias by: 
$$bias = W.X - y$$

In [ ]:
a = torch.inverse(torch.mm(X_train.T, X_train))
b = torch.mm(a, X_train.T)

w = torch.mm(b, y_train.view(-1, 1))
bias = (y_train - torch.mm(X_train, w).view(-1))[0]

print(f'Coefficients: {list(zip(feature_cols, w.view(-1).numpy()))}')
print(f'Intercept: {bias:.3f}')